In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import argparse
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import cross_validate
import os
import sys
from pathlib import Path
from datetime import datetime, timedelta
# from xgboost import XGBClassifier

from tqdm import tqdm

# pd.options.display.max_columns = None



# PREPROCESS DATA

In [ ]:
fdir_raw = Path("/home/ar3/Documents/PYTHON/RNASeqAnalysis/data/raw/")
data = pd.read_csv(fdir_raw/'Geuvadis.all.csv', index_col=0)
# data.rename(columns={"Unnamed: 0": "trascripts"})
data = data.T
data

In [ ]:
data_header = pd.read_csv(fdir_raw/'Geuvadis.SraRunTable.txt',index_col=0)
data_header = data_header[['Sex', 'Experimental_Factor:_population (exp)']]
# # data_header.set_index('Run', inplace=True)
data_header.head()

# data_header.columns

In [ ]:
def filter_by_non_zero_median(df):
    print(df.shape)
    df_median = df.median()
    if (df_median == 0).any():
        cols_to_drop = df.columns[df_median == 0]
        print(len(cols_to_drop),
              " features will be removed, due to a zero median value")
        df = df.drop(columns=cols_to_drop)
        print("Current dataset size: ", df.shape)
        return df

    print("Zero median columns aren't found")
    print('Dataset shape: ', df.shape)
    return df

data = filter_by_non_zero_median(data).astype('float32')
data

In [ ]:
from scipy.stats import pointbiserialr


X_corr = data
y_corr = data_header['Sex'].loc[data.index]

for c in tqdm(X_corr.columns):
    corr, pvalue = pointbiserialr(X_corr[c], LabelEncoder().fit_transform(y_corr.values))
    if np.abs(corr) > 0.5:
        print(c)

data.values[:, 0].shape, data_header['Sex'].loc[data.index].values.shape
# data['Sex']

In [ ]:
numerical_cols = data.iloc[:1].select_dtypes(include=[np.number]).columns
data = data.replace(0, 1e-6)
data = np.log(data)
data

In [ ]:
def filter_by_cv(df, threshold):
    cv = df.std() / df.mean()
    # print(cv)
    low_cv_cols = cv[cv < threshold].index

    if len(low_cv_cols) > 0:
        print(f"{len(low_cv_cols)} features have coefficient of variation below {threshold} and will be removed.")
        df = df.drop(columns=low_cv_cols)
    else:
        print("No features found with coefficient of variation below the threshold.")
    print(f"Current amount of features is {len(df.columns)}")
    return df

cv_threshold = 0.7
data = filter_by_cv(data, cv_threshold)
data

In [ ]:
data.quantile(q=0.5)

In [ ]:
fdir_processed = Path("/home/ar3/Documents/PYTHON/RNASeqAnalysis/data/interim")
data = data.astype(np.float32)
data['Sex'] = data_header['Sex']
data.to_csv(fdir_processed/'geuvadis.preprocessed.XY.csv')
# dataset

## Remove transcripts located on sex chromosome

In [ ]:
from gtfparse import read_gtf

gtf_data = read_gtf(fdir_raw/'gencode.v44.annotation.gtf')
gtf_data = gtf_data.to_pandas()


In [ ]:
gtf_data = gtf_data[['seqname', 'transcript_id']]
transcripts_x = gtf_data.loc[gtf_data['seqname'] == 'chrX', 'transcript_id']
transcripts_y = gtf_data.loc[gtf_data['seqname'] == 'chrY', 'transcript_id']
transcripts_x = transcripts_x.unique() 
transcripts_y = transcripts_y.unique()

pd.Series(transcripts_x).to_csv(fdir_processed/"transcripts_X.csv")
pd.Series(transcripts_y).to_csv(fdir_processed/"transcripts_Y.csv")


In [ ]:

transcripts_x = pd.read_csv(fdir_processed/"transcripts_X.csv", index_col=0).values.ravel().tolist()
transcripts_y = pd.read_csv(fdir_processed/"transcripts_Y.csv", index_col=0).values.ravel().tolist()


data_noX = data.drop(columns=data.columns.intersection(transcripts_x))
data_noY = data.drop(columns=data.columns.intersection(transcripts_y))
data_noXY = data_noY.drop(columns=data.columns.intersection(transcripts_x))

# data.columns.intersection(transcripts_y)

data_noX.to_csv(fdir_processed/'geuvadis.preprocessed.Y.csv')
data_noY.to_csv(fdir_processed/'geuvadis.preprocessed.X.csv')
data_noXY.to_csv(fdir_processed/'geuvadis.preprocessed.csv')


---


# LOAD PREPROCESSED DATA

In [ ]:
fdir_processed = Path("/home/ar3/Documents/PYTHON/RNASeqAnalysis/data/interim")

# sex = "." + 'Y'
sex = ""

data = pd.read_csv(fdir_processed/f'geuvadis.preprocessed{sex}.csv', index_col=0)
data

In [ ]:
# pointbiserialr(X_corr["MSTRG.563.1"], LabelEncoder().fit_transform(y_corr.values))

# # sns.boxplot(data[["MSTRG.563.1", 'Sex']], x='Sex', y="MSTRG.563.1")
# # X_corr["MSTRG.563.1"].hist()

In [ ]:
X = data.drop(columns=['Sex'])
y = data['Sex']

test_size = 0.2
random_state = 42

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, shuffle=True)


train_scaler = StandardScaler().fit(X_train)
test_scaler = StandardScaler().fit(X_test)

X_train = train_scaler.transform(X_train) 
X_test = test_scaler.transform(X_test)


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder().fit(y_train)
y_train = label_encoder.transform(y_train)
# y_train

In [ ]:
X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train)


## Train model


In [ ]:
n_threads = 6

# CBC = CatBoostClassifier(loss_function='MultiClass',
#                          od_pval=0.05,
#                          thread_count=n_threads,
#                          task_type="CPU",
#                          iterations=500,
#                          learning_rate=0.03
#                          #  devices='0'
#                          )

# CBC.fit(X_train_, y_train_, eval_set=(X_val, y_val), 
#         verbose=False, 
#         use_best_model=True, 
#         plot=True, 
#         early_stopping_rounds=20)

# model = CBC

model = XGBClassifier(early_stopping_rounds=20)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)])



In [ ]:
ml_models_fdir = Path("/home/ar3/Documents/PYTHON/RNASeqAnalysis/models")

if not (ml_models_fdir/'catboost').is_dir():
    (ml_models_fdir/'catboost').mkdir()

saved_model_filename = f"geuvadis_{sex}.cbm"
model.save_model(fname=ml_models_fdir/'catboost'/saved_model_filename, 
               format='cbm', export_parameters=None, pool=None)


In [ ]:
# sex = 'XY'
saved_model_filename = f"geuvadis_{sex}.cbm"
model.load_model(fname=ml_models_fdir/'catboost'/saved_model_filename)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,  RocCurveDisplay

pred = model.predict(X_test)
pred_prob = model.predict_proba(X_test)

ConfusionMatrixDisplay(
    confusion_matrix(label_encoder.transform(y_test),
                 pred)
).plot()

RocCurveDisplay.from_predictions(
    label_encoder.transform(y_test), pred_prob[:, 1]
)

In [ ]:
feature_importance_df = pd.DataFrame({
    'Feature': data.drop(columns='Sex').columns,
    'Importance': model.feature_importances_
    # 'Importance': XGB.feature_importances_
})
feature_importance_df = feature_importance_df.sort_values(
    by='Importance', ascending=False)


feature_importance_df.iloc[:10]

In [ ]:
import shap
shap.initjs()

explainer = shap.TreeExplainer(model)
shap_values = explainer(pd.DataFrame(X_train_, columns=data.columns.drop('Sex')))

# visualize the first prediction's explanation

# shap.plots.force(explainer.expected_value, shap_values)

# shap.dependence_plot("MSTRG.563.1", shap_values.values, X_train_, interaction_index="HouseAge")
shap.plots.beeswarm(shap_values)


In [ ]:
data['Sex'], label_encoder.transform(y)